In [1]:
# only permit max cut

canonical_order = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz',  'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB']
def reorder(df, canonical_order=canonical_order, by='dataset', extras=['dataset','gen_n', 'gen_n_max'], secondary='gen_n', columns=None):
    df['dataset_name_order'] = df[by].map({name: i for i, name in enumerate(canonical_order)})
    if secondary is not None:
        df = df.sort_values(by=['dataset_name_order', secondary])
    else:
        df = df.sort_values(by=['dataset_name_order'])

    df.drop('dataset_name_order', axis =1, inplace=True)

    if columns:
        return df[columns ]
    else:
        return df

In [2]:
import os
os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs')

['legacy',
 'Testing',
 '230928_snapshot',
 'LiftMP_runs',
 '230924_hparam2',
 '230927_snapshot',
 '230928_runs',
 '230924_hparam',
 '230924_hparam_TU_multiarch',
 '230924_hparam_TU',
 '230926_finetune_ER_runs']

In [3]:
print(list(os.path.join('230928_runs', x) for x in os.listdir('/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/230928_runs')))

['230928_runs/230925_TUsmall_GAT_VC', '230928_runs/230925_TUsmall_GIN_cut', '230928_runs/230925_generated_preset_cut', '230928_runs/230925_TUsmall_GAT_cut', '230928_runs/230925_TUsmall_liftMP_cut', '230928_runs/230925_TUsmall_GCNN_VC', '230928_runs/230925_TUsmall_GCNN_cut', '230928_runs/230925_generated_liftMP_VC', '230928_runs/230925_generated_preset_VC', '230928_runs/230925_TUsmall_liftMP_VC', '230928_runs/230925_TUsmall_GatedGCNN_VC', '230928_runs/230925_TUsmall_VC_32', '230928_runs/230925_TUlarge_liftMP_cut', '230928_runs/230925_TUlarge_liftMP_VC', '230928_runs/230925_generated_liftMP_cut', '230928_runs/230925_TUsmall_GatedGCNN_cut']


In [4]:
run_folders = ['LiftMP_runs', '230927_snapshot/230925_TUsmall_GAT_VC', '230927_snapshot/230925_TUsmall_GIN_VC', 
               '230927_snapshot/230925_generated_preset_cut', '230927_snapshot/230925_TUlarge_all_cut', '230927_snapshot/230925_TUsmall_GCNN_VC',
                 '230927_snapshot/230925_generated_liftMP_VC', '230927_snapshot/230925_TUlarge_all_VC', '230927_snapshot/230925_generated_preset_VC', 
                 '230927_snapshot/230925_TUsmall_liftMP_VC', '230927_snapshot/230925_TUsmall_GatedGCNN_VC', '230927_snapshot/230925_generated_liftMP_cut']

run_folders = ['230928_snapshot/230925_TUsmall_GAT_VC', '230928_snapshot/230925_TUsmall_GIN_cut', 
               '230928_snapshot/230925_generated_preset_cut', '230928_snapshot/230925_TUsmall_GAT_cut', 
               '230928_snapshot/230925_TUsmall_liftMP_cut', '230928_snapshot/230925_TUsmall_GCNN_VC', 
               '230928_snapshot/230925_TUsmall_GCNN_cut', '230928_snapshot/230925_generated_liftMP_VC', 
               '230928_snapshot/230925_generated_preset_VC', '230928_snapshot/230925_TUsmall_liftMP_VC', '230928_snapshot/230925_TUsmall_GatedGCNN_VC', 
               '230928_snapshot/230925_TUsmall_VC_32', '230928_snapshot/230925_generated_liftMP_cut', '230928_snapshot/230925_TUsmall_GatedGCNN_cut']

run_folders = ['230928_runs/230925_TUsmall_GAT_VC', '230928_runs/230925_TUsmall_GIN_cut', 
               '230928_runs/230925_generated_preset_cut', '230928_runs/230925_TUsmall_GAT_cut', 
               '230928_runs/230925_TUsmall_liftMP_cut', '230928_runs/230925_TUsmall_GCNN_VC', 
               '230928_runs/230925_TUsmall_GCNN_cut', '230928_runs/230925_generated_liftMP_VC', '230928_runs/230925_generated_preset_VC', 
               '230928_runs/230925_TUsmall_liftMP_VC', '230928_runs/230925_TUsmall_GatedGCNN_VC', '230928_runs/230925_TUsmall_VC_32', 
               '230928_runs/230925_TUlarge_liftMP_cut', '230928_runs/230925_TUlarge_liftMP_VC', '230928_runs/230925_generated_liftMP_cut', '230928_runs/230925_TUsmall_GatedGCNN_cut']



In [26]:
import json
import pandas as pd
import numpy as np

folder_path = '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs'
model_list = [os.path.join(folder_path, run_folder, x) for run_folder in run_folders for x in os.listdir(os.path.join(folder_path, run_folder))  ]


rows = []
numtimes = 0

errored = []
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        if model_args['problem_type'] != 'max_cut':
            continue
        
        losses = np.load(os.path.join(model_folder, 'valid_scores.npy'))
        test_losses = np.load(os.path.join(model_folder, 'test_scores.npy'))
        modeldict = model_args #{x: model_args[x] for x in params}
        modeldict['max_valid_score'] = max(losses)
        modeldict['max_valid_epoch'] = np.argmax(losses)
        modeldict['scores'] = test_losses[np.argmax(losses)]
        modeldict['baseline'] = False


        valid_score_from_file = -np.inf
        time_per_pred = None

        for prefix in ['retest_best', 'retest_last']:
            scorefile = [x for x in os.listdir(model_folder) if x.startswith(prefix)]
            #assert(len(scorefile) <=1)
            if len(scorefile) >= 1:
                times, scores = np.load(os.path.join(model_folder, scorefile[0]))
                valid_score = np.average(scores)
                if valid_score > valid_score_from_file:
                    valid_score_from_file = valid_score
                time_per_pred = np.average(times)
        
        if valid_score_from_file > modeldict['max_valid_score']:
            modeldict['scores'] = valid_score_from_file
        modeldict['time_per_pred'] = time_per_pred
        #if time_per_pred == None and valid_score_from_file != -np.inf:
        #    print(time_per_pred, valid_score_from_file)
        
        rows.append(modeldict)
        
    except Exception as e:
        print(f'{e} is wrong w/ {model_folder}')
        errored.append(model_folder)

In [6]:
numtimes

0

In [7]:
# load in baselines
baselines = pd.read_csv('mc_baseline_scores.csv')

gen_n_dict = dict(zip([50, 100, 400], [[50, 100], [100, 200],[400, 500]]))

# unwind them 
for i, baseline in baselines.iterrows():
    #print(baseline.index)
    for col in baselines.keys():
        if col == 'Unnamed: 0':
            continue
        if baseline['Unnamed: 0'] == 'edge count':
            continue
        ds = col
        gen_n = np.nan
        if len(col.split('@@')) > 1:
            ds, gen_n = col.split('@@')
            gen_n = gen_n_dict[int(gen_n)]

        row = {'dataset': ds, 'gen_n': gen_n, 'model_type': baseline['Unnamed: 0'], 'scores': baseline[col], 'baseline': True}
        #print(row)
        rows.append(row)


In [8]:
len(errored)

0

In [9]:
df = pd.DataFrame.from_records(rows)

In [28]:
sum(~df.time_per_pred.isna())

304

In [11]:
df['gen_n_list'] = df.gen_n
df['gen_n_max'] = df.gen_n_list.apply(lambda x: int(x[1]) if isinstance(x,list) else x)
df.gen_n = df.gen_n.apply(lambda x: int(x[0]) if isinstance(x,list) else x)


In [12]:
df.keys()
# 'problem_type', 'seed',  'prefix', 'RB_n', 'RB_k', 'log_dir',
relevant_keys = [ 'model_type', 'num_layers',
       'repeat_lift_layers', 'num_layers_project', 'rank', 'vc_penalty', 'gen_n',
       'dataset', 'infinite',  'positional_encoding', 'pe_dimension',
       'max_valid_score', 'max_valid_epoch',
       'scores']

df

import matplotlib.pyplot as plt
import seaborn as sns
sns.histplot(df[df.infinite==False].max_valid_epoch)
plt.title('the step (x1000) at which the lowest validation score is achieved (non-infinite)')
plt.xlabel("step (x1000)")
plt.ylabel('count')
plt.show()
df[(df.max_valid_epoch > 90) & (df.infinite==False)][relevant_keys] #.to_csv('230925_checkup_late_bloomers.csv')

sns.histplot(df[df.infinite!=False].max_valid_epoch, bins=range(20))
plt.title('the step (x1000) at which the lowest validation score is achieved (infinite)')
plt.xlabel("step (x1000)")
plt.ylabel('count')
plt.show()
df[(df.max_valid_epoch > 18) & (df.infinite!=False)][relevant_keys] #.to_csv('230925_checkup_late_bloomers.csv')

df.loc[df[df.baseline == False].groupby(['dataset', 'gen_n'])['scores'].idxmax()][relevant_keys]

In [17]:
df.gen_n = df.gen_n.fillna(100)
df.gen_n_max = df.gen_n_max.fillna(100)

from collections import Counter
Counter(df.dataset)

In [19]:
# test models

models_for_test = []

In [20]:
dataset_by_arc = pd.DataFrame()
dss = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz']
mts = ['SDP proj', 'gurobi_2.0', 'gurobi_4.0', 'gurobi_8.0', 'vertex count']



for (mt, ds, gen_n), group in df[(df.infinite == False) | (df.dataset == 'ErdosRenyi')].groupby(['model_type', 'dataset', 'gen_n']):
    if mt in mts:
        continue
    if all(group['scores'].isna()):
        continue

    if ds not in dss:
        gen_n = ""

    #print(mt, ds, gen_n)
    if ds in dss:
        dataset_by_arc.at[f'{ds}, {gen_n}', mt] = f'{group["scores"].max():0.1f} ({df.loc[group["scores"].idxmax()]["time_per_pred"]*1000:0.0f})'
        dataset_by_arc.at[f'{ds}, {gen_n}', 'gen_n'] = gen_n
        dataset_by_arc.at[f'{ds}, {gen_n}', 'dataset'] = ds
    else: 
        dataset_by_arc.at[f'{ds}', mt] = f'{group["scores"].max():0.1f} ({df.loc[group["scores"].idxmax()]["time_per_pred"]*1000:0.0f})'
        dataset_by_arc.at[f'{ds}', 'dataset'] = ds

    try:
        models_for_test.append((df.loc[group['scores'].idxmax()]['log_dir'], df.loc[group['scores'].idxmax()]['dataset'], df.loc[group['scores'].idxmax()]['gen_n'])) 
    except Exception as e:
        print(f'{e}')



#dataset_by_arc['dataset'] = dataset_by_arc.index
reorder(dataset_by_arc[[k for k in dataset_by_arc.keys() if k not in ['SDP lift', 'edge count']]], by='dataset', columns = ['dataset', 'gen_n',  'GAT', 
                                                                                                                            'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']).to_csv('Table1_maxcut.csv', index=False) #.style.highlight_max(color = 'green', axis = 1)

In [21]:
dataset_by_arc = pd.DataFrame()
mts = ['GAT', 'GCNN', 'GIN', 'GatedGCNN']
dss = ['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz']

for (mt, ds, gen_n, gen_nmax), group in df[~df.model_type.isin(mts)].groupby(['model_type', 'dataset', 'gen_n', 'gen_n_max']):


    if all(group['scores'].isna()):
        continue
    if ds not in dss:
        gen_n = ""
        gen_nmax = ""
    #print(mt, ds, gen_n)
    if ds in dss:
        k = f'{ds}_{gen_n}'
    else:
        k = ds
    dataset_by_arc.at[k, 'Type'] = ds
    dataset_by_arc.at[k, 'Nmin'] = gen_n
    dataset_by_arc.at[k, 'Nmax'] = gen_nmax
    if mt != 'LiftMP':
        dataset_by_arc.at[k, mt] = group['scores'].max()
    else:
        dataset_by_arc.at[k, mt] = f'{group["scores"].max():0.1f} ({df.loc[group["scores"].idxmax()]["time_per_pred"]*1000:0.0f})'
    try:
        models_for_test.append((df.loc[group['scores'].idxmax()]['log_dir'], df.loc[group['scores'].idxmax()]['dataset'], df.loc[group['scores'].idxmax()]['gen_n'])) 
    except Exception as e:
        print(f'{e}')

#dataset_by_arc.rename(columns={'Nikos': 'CustomLiftProject'}, inplace=True)
reorder(dataset_by_arc[[k for k in dataset_by_arc.keys() if k not in [ 'SDP lift', 'vertex count']]], by='Type', secondary='Nmin').round(1).to_csv('Table2_maxcut.csv') #.style.highlight_max(color = 'green', axis = 1)

In [22]:
for x in models_for_test:
    print(x)

('training_runs/230925_TUsmall_GAT_cut/paramhash:229fdd4defd5beb6d5299b78886340673e646af2288d38d62612ac1d54d79e35', 'COLLAB', 100.0)
('training_runs/230925_TUsmall_GAT_cut/paramhash:13208a7aa1df13fe6517cb5e1ee7d27df9e4436f2cfbce9ddf35ffee6440d8a5', 'ENZYMES', 100.0)
('training_runs/230925_generated_preset_cut/paramhash:98627c0c9f1729b615470a023c62c47b0d4265e0180843f09678d0dccf98f1e3', 'ErdosRenyi', 50.0)
('training_runs/230925_generated_preset_cut/paramhash:29022497978cacef4919cb917e4407ea28c15be66dbeb1afebecce605652db9d', 'ErdosRenyi', 100.0)
('training_runs/230925_generated_preset_cut/paramhash:45aa5663131bdb65f33407dc287ffd53552b8b7bc847585738935981839e6d70', 'ErdosRenyi', 400.0)
('training_runs/230925_TUsmall_GAT_cut/paramhash:6bb1d7ca49813746e5f0b172f02d8f9745b818b7f29c3f0e50229f0eaaa7343d', 'IMDB-BINARY', 100.0)
('training_runs/230925_TUsmall_GAT_cut/paramhash:a4ffb64d114dba179d14cfea0af4813f48fe0b9379f65ca45e2d13450ab46f2e', 'MUTAG', 100.0)
('training_runs/230925_TUsmall_GAT_cut